In [9]:
# нью рок нью джаз
import numpy as np
import pandas as pd

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

imputer = SimpleImputer(strategy="most_frequent")

In [11]:
path_train = "train"

wag_prob = pd.read_parquet(path_train + "/wagons_probeg_ownersip.parquet").convert_dtypes()
target = pd.read_csv(path_train + '/target/y_train.csv').convert_dtypes()
to_predict = pd.read_csv(path_train + '/target/y_predict.csv').convert_dtypes()
wag_param = pd.read_parquet(path_train + "/wag_params.parquet").convert_dtypes()
# добавляем kri_izm
kri_izm = pd.read_parquet(path_train + '/kti_izm.parquet').convert_dtypes()

In [12]:
target

,wagnum,month,target_month,target_day
0,33361,2023-01-01,0,0
1,33364,2023-01-01,0,0
2,33366,2023-01-01,0,0
3,33358,2023-01-01,0,0
4,33349,2023-01-01,0,0
...,...,...,...,...
203848,25045,2022-12-01,0,0
203849,27156,2022-12-01,0,0
203850,21361,2022-12-01,0,0
203851,8061,2022-12-01,0,0


In [13]:
wag_prob

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
1,2022-08-02,33361,7243,0,1,1,0,8
2,2022-08-03,33361,6990,0,1,1,0,8
3,2022-08-04,33361,6347,0,1,1,0,8
4,2022-08-05,33361,6027,0,1,1,0,8
...,...,...,...,...,...,...,...,...
9249584,2022-12-20,33350,35062,0,1,0,0,12
9249585,2022-12-21,33350,35062,0,1,0,0,12
9249586,2022-12-22,33350,35062,0,1,0,0,12
9249587,2022-12-23,33350,35062,0,1,0,0,12


In [14]:
train_target = target

train = wag_prob


path_test = 'test'
test_target = pd.read_csv(path_test + '/target/y_test.csv').convert_dtypes().drop("month", axis=1)
test = wag_prob


In [15]:
train.head()

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
1,2022-08-02,33361,7243,0,1,1,0,8
2,2022-08-03,33361,6990,0,1,1,0,8
3,2022-08-04,33361,6347,0,1,1,0,8
4,2022-08-05,33361,6027,0,1,1,0,8


In [16]:
def get_dislok(path=path_train + "/dislok_wagons.parquet"):

    dislok_wagons = pd.read_parquet(path).convert_dtypes()
    
    dis_train = dislok_wagons[dislok_wagons["plan_date"] < "2023-02-01"]
    february_date = "2023-02-01"
    base_date = pd.to_datetime(february_date)
    days_from_kap_mean = (base_date - dis_train["date_kap"]).dt.days.mean()
    days_from_dep_mean = (base_date - dis_train["date_dep"]).dt.days.mean()
    
    dis_train["days_from_kap"] = (pd.to_datetime(february_date) - dis_train["date_kap"]).dt.days
    dis_train["days_from_dep"] = (pd.to_datetime(february_date) - dis_train["date_dep"]).dt.days
    
    # Fill missing values with mean
    dis_train['days_from_kap'] = dis_train['days_from_kap'].fillna(-1)
    dis_train['days_from_dep'] = dis_train['days_from_dep'].fillna(-1)
    dis_train["days_to_date_pl_rem"] = (base_date - dis_train["date_pl_rem"]).dt.days
    
    dis_train = dis_train.groupby('wagnum').agg({
        'plan_date': ['last'], 
        'days_from_kap': lambda x: x[x != -1].iloc[0] if (x != -1).any() else days_from_kap_mean,
        'days_from_dep': lambda x: x[x != -1].iloc[0] if (x != -1).any() else days_from_dep_mean,
        'days_to_date_pl_rem': lambda x: x[x.notnull()].iloc[-1] if x.notnull().any() else None,
        'date_pl_rem': lambda x: x[x.notnull()].iloc[-1] if x.notnull().any() else None, # поменять
        'distance': 'last',
        'isload': 'last',
        'ost_prob': 'last'
    })
    
    dis_train.columns = ['plan_date', 'days_from_kap', 'days_from_dep', 'days_to_date_pl_rem',
                     'date_pl_rem', 'distance', 'isload', 'ost_prob']
    return dis_train['days_to_date_pl_rem']
    #return dis_train[['']]

In [17]:
def prepare_data(group_data, wag_param, target=None, columns_to_drop=None,
                 columns_to_drop_wag_param=None, columns_to_log=None,
                 imputer_strategy='most_frequent', without_wagnum=True):
    # Group and aggregate the data
    if columns_to_drop_wag_param is None:
        columns_to_drop_wag_param = ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"]

    features = group_data.groupby("wagnum").agg(
        {
            "ost_prob": ["mean", "last", "max", 'std', 'min'],
            "manage_type": "last",
            "rod_id": "last",
            "reestr_state": "last",
        }
    )
    features.columns = ["_".join(col).strip() for col in features.columns.values]
    features = features.reset_index()

    # Drop unnecessary columns from wag_param
    m_data = wag_param.drop(["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1)

    # Merge the aggregated features with the wag_param
    merged = features.merge(m_data, on="wagnum", how="inner")
    
    merged = merged.merge(get_dislok(), on='wagnum', how='inner')
    
    # If a target dataframe is provided, merge it as well
    if target is not None:
        merged = target.merge(merged, on="wagnum", how="left")

    # Define and apply an imputer for missing values
    imputer = SimpleImputer(strategy=imputer_strategy)

    column_names = merged.columns
    merged = imputer.fit_transform(merged)
    
    merged = pd.DataFrame(merged, columns=column_names)

    if columns_to_log is not None:
        for column in columns_to_log:
            merged[column] = np.log(merged[column])

    if columns_to_drop is not None:
        merged = merged.drop(columns_to_drop, axis=1)
        

    
    # Drop the 'wagnum' column to prepare the feature matrix
    if without_wagnum:
        merged = merged.drop(["wagnum"], axis=1)

    return merged

In [18]:


# Define the columns to drop from wag_param
columns_to_drop = ['tippogl',
                   'tormoz',
                   'norma_km',
                   'kuzov',
                   'telega',
                   'cnsi_probeg_dr',
                   'manage_type_last',
                   'ownertype',
                   'cnsi_probeg_kr',
                   'reestr_state_last', 'zavod_build',
                   'cnsi_gruz_capacity',
                   'rod_id',
                   'gruz',
                   'rod_id_last']

#columns_to_log = ['ost_prob_mean', 'ost_prob_last', 'ost_prob_max', 'tara']
# Prepare training data
X_train = prepare_data(train, wag_param, train_target, columns_to_drop)

# Prepare testing data
X_test = prepare_data(test, wag_param, test_target, columns_to_drop)


ValueError: Cannot use most_frequent strategy with non-numeric data:
could not convert string to float: '2023-01-01'

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train.to_csv('x_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)

In [ ]:

y_month_name = "target_month"
y_day_name = "target_day"
target_names = [y_month_name, y_day_name]

In [ ]:
print("Training for month and day prediction")

logreg_month = LogisticRegression()
logreg_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
logreg_day = LogisticRegression()
logreg_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

rf_month = RandomForestClassifier()
rf_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
rf_day = RandomForestClassifier()
rf_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])




In [ ]:
# Подсчитаем количество примеров каждого класса
class_counts = X_train[y_month_name].value_counts()
# Например, если у нас два класса, то вес для каждого класса может быть обратно пропорционален его частоте.
class_weights = class_counts.sum() / (len(class_counts) * class_counts)
cb_month = CatBoostClassifier(random_seed=42,
                              auto_class_weights='Balanced',
                              #eval_metric='F1', 
                              )
cb_month.fit(
    X_train.drop(target_names, axis=1),
    X_train[y_month_name],
    eval_set=(X_test.drop(target_names, axis=1), X_test[y_month_name]),
    verbose=False, )

# Подсчитаем количество примеров каждого класса
class_counts = X_train[y_day_name].value_counts()
# Например, если у нас два класса, то вес для каждого класса может быть обратно пропорционален его частоте.
class_weights = class_counts.sum() / (len(class_counts) * class_counts)
cb_day = CatBoostClassifier(random_seed=42,
                            auto_class_weights='Balanced',
                            #eval_metric='F1',
                            iterations=1000)
cb_day.fit(
    X_train.drop(target_names, axis=1),
    X_train[y_day_name],
    eval_set=(X_test.drop(target_names, axis=1), X_test[y_day_name]),
    verbose=False,
    use_best_model=False
)

In [ ]:
y_pred_logreg_month = logreg_month.predict(X_test.drop(target_names, axis=1))
y_pred_rf_month = rf_month.predict(X_test.drop(target_names, axis=1))
y_pred_cb_month = cb_month.predict(X_test.drop(target_names, axis=1))

y_pred_logreg_day = logreg_day.predict(X_test.drop(target_names, axis=1))
y_pred_rf_day = rf_day.predict(X_test.drop(target_names, axis=1))
y_pred_cb_day = cb_day.predict(X_test.drop(target_names, axis=1))

print("Month score:")
f1_logreg_month = f1_score(X_test[y_month_name], y_pred_logreg_month)
print(f"Logistic Regression month F1 Score: {f1_logreg_month}")
f1_rf_month = f1_score(X_test[y_month_name], y_pred_rf_month)
print(f"Random Forest month F1 Score: {f1_rf_month}")
f1_cb_month = f1_score(X_test[y_month_name], y_pred_cb_month)
print(f"CatBoost month F1 Score: {f1_cb_month}")

print("Day score:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day)
print(f"CatBoost day F1 Score: {f1_cb_day}")

In [ ]:
def correct_days_with_month(day_array, month_array):
    corrected_day = day_array.copy()
    corrected_day[month_array == 0] = 0
    return corrected_day


y_pred_logreg_day_corrected = correct_days_with_month(y_pred_logreg_day, y_pred_logreg_month)
y_pred_rf_day_corrected = correct_days_with_month(y_pred_rf_day, y_pred_rf_month)
y_pred_cb_day_corrected = correct_days_with_month(y_pred_cb_day, y_pred_cb_month)

print("Day score after applying month info to day:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day_corrected)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day_corrected)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day_corrected)
print(f"CatBoost day F1 Score: {f1_cb_day}")


In [ ]:
X_test.drop(target_names, axis=1).columns

In [ ]:
feature_importances = cb_month.get_feature_importance()
feature_names = X_test.drop(target_names, axis=1).columns
importance_dict = {feature_names[i]: importance for i, importance in enumerate(feature_importances)}
sorted_f_imp = dict(sorted(importance_dict.items(), key=lambda item: item[1], reverse=True))

In [ ]:
sorted_f_imp

In [ ]:
list(sorted_f_imp.keys())

In [ ]:
wag_param.columns

In [ ]:
# import os
# import h2o
# import xgboost as xgb
# import pandas as pd
# from h2o.automl import H2OAutoML
# 
# h2o.init()
# 
# model_path = ""
# model_file = os.path.join(model_path, "StackedEnsemble_AllModels_1_AutoML_2_20231111_112553")
# train = h2o.H2OFrame(X_train)
# test = h2o.H2OFrame(X_test)
# train[y_month_name] = train[y_month_name].asfactor()
# test[y_month_name] = test[y_month_name].asfactor()
# 
# x = train.columns
# y = y_month_name
# x.remove(y)
# 
# # if os.path.exists(model_file):
# #     print("sdfsd")
# #     aml = h2o.load_model(model_file)
# # else:
# aml = H2OAutoML(max_models=15, seed=42)
# aml.train(x=x, y=y, training_frame=train)
# 
# h2o.save_model(aml.leader, path=model_path)
# 
# lb = aml.leaderboard
# print(lb.head(rows=lb.nrows))

In [ ]:
# y_pred = aml.predict(test).as_data_frame().squeeze()
# result = f1_score(X_test[y_month_name], y_pred["predict"])
# print(f"F1: {result}")


In [ ]:
# train = h2o.H2OFrame(X_train)
# test = h2o.H2OFrame(X_test)
# train[y_day_name] = train[y_day_name].asfactor()
# test[y_day_name] = test[y_day_name].asfactor()
# x = train.columns
# y = y_day_name
# x.remove(y)
# 
# # if os.path.exists(model_file):
# #     print("sdfsd")
# #     aml = h2o.load_model(model_file)
# # else:
# aml_day = H2OAutoML(max_models=15, seed=42)
# aml_day.train(x=x, y=y, training_frame=train)
# 
# h2o.save_model(aml_day.leader, path=model_path)
# 
# lb = aml_day.leaderboard
# print(lb.head(rows=lb.nrows))

In [ ]:
# y_pred_day = aml_day.predict(test).as_data_frame().squeeze()
# result_day = f1_score(X_test[y_day_name], y_pred_day["predict"])
# print(f"F1: {result_day}")

## Меняем train и test для того чтобы сделать корректный submission

In [ ]:
# def prepare_data_for_submission(group_data, wag_param, target=None, columns_to_drop=None,
#                                 columns_to_drop_wag_param=None,columns_to_log=None,
#                                 imputer_strategy='most_frequent', without_wagnum=True):
#     # Group and aggregate the data
#     if columns_to_drop_wag_param is None:
#         columns_to_drop_wag_param = ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"]
# 
#     features = group_data.groupby("wagnum").agg(
#         {
#             "ost_prob": ["mean", "last", "max", 'var'],
#             "manage_type": "last",
#             "rod_id": "last",
#             "reestr_state": "last",
#         }
#     )
#     features.columns = ["_".join(col).strip() for col in features.columns.values]
#     features = features.reset_index()
# 
#     # Drop unnecessary columns from wag_param
#     m_data = wag_param.drop(["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1)
# 
#     # Merge the aggregated features with the wag_param
#     merged = features.merge(m_data, on="wagnum", how="inner")
# 
#     # If a target dataframe is provided, merge it as well
#     if target is not None:
#         merged = target.merge(merged, on="wagnum", how="left")
# 
#     # Define and apply an imputer for missing values
#     imputer = SimpleImputer(strategy=imputer_strategy)
# 
#     column_names = merged.columns
#     merged = imputer.fit_transform(merged)
#     merged = pd.DataFrame(merged, columns=column_names)
#     
#     if columns_to_log is not None:
#         for column in columns_to_log:
#             merged[column] = np.log(merged[column])
# 
#     if columns_to_drop is not None:
#         merged = merged.drop(columns_to_drop, axis=1)
# 
#     # Drop the 'wagnum' column to prepare the feature matrix
#     #X_data = merged.drop(["wagnum"], axis=1)
# 
#     return merged

In [ ]:
wag_param_submission = pd.read_parquet(path_test + '/wag_params.parquet').convert_dtypes()

wag_prob_submission = pd.read_parquet('test/wagons_probeg_ownersip.parquet').convert_dtypes()
wag_prob_submission = pd.concat([wag_prob, wag_prob_submission])

In [ ]:
x_submission = prepare_data(wag_prob_submission, wag_param_submission, columns_to_drop=columns_to_drop, without_wagnum=False)
x_submission['wagnum'] = x_submission['wagnum'].astype(int)

In [ ]:
x_submission[['ost_prob_last', 'wagnum']].head(15)

In [ ]:
x_submission.info()

In [ ]:
submission_info = pd.read_csv('train/target/y_predict.csv').convert_dtypes()


In [ ]:
submission_info.info()

In [ ]:
submission_info = submission_info.merge(x_submission, how='left', on='wagnum')

In [ ]:
submission_info

In [ ]:
# submission_h2o = h2o.H2OFrame(submission_info.drop(['wagnum', 'month'], axis=1))
# predictions = aml.predict(submission_h2o).as_data_frame().squeeze()
# submission_info['target_month'] = predictions['predict']
submission_info['target_month'] = cb_month.predict(submission_info.drop(['wagnum', 'month'], axis=1)).astype(int)
submission_info['target_day'] = cb_day.predict(submission_info.drop(['wagnum', 'month'], axis=1)).astype(int)

In [ ]:
submission_info['target_day'] = correct_days_with_month(submission_info['target_day'], submission_info['target_month'])
submission_info.head(15)

In [ ]:
submission_info[['wagnum', 'month', 'target_month', 'target_day']].to_csv('submission.csv', index=False)

In [ ]:
# h2o.save_model(aml.leader, path=f"saved_models/h20_{result}")
# h2o.shutdown()

In [ ]:
# feature_importance = logreg.coef_
# 
# avg_feature_importance = np.mean(np.abs(feature_importance), axis=0)
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": avg_feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [ ]:
# feature_importance = rf.feature_importances_
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [ ]:
# import shap
# 
# explainer = shap.TreeExplainer(rf)
# shap_values = explainer.shap_values(X_train)
# shap.summary_plot(shap_values, X_train, plot_type="bar")